# **EXECUTIVE SUMMARY**    
The objective of this project is to build a machine learning model that will predict taxicab trip fare before riding based on 2022 NYC Yellow Cab trip record data (https://data.cityofnewyork.us/Transportation/2022-Yellow-Taxi-Trip-Data/qp3b-zxtp).
To that end, I will demonstrate the phases of the data science pipeline:
1. **[Random Sampling](#sampl)** Make a Random Sample from the large original CSV file with (39.7 Milions rows & Size More than 3.5 GB) to make the analysis and modeling process more efficient: 
2. **[Preprocessing](#prep)**: Clean and transform the data into a usable format for analysis.
3. **[Exploratory Analysis](#eda)**: Perform exploratory analysis to identify the best features to be used for modeling.  And Test the Models Assumptions to choose suitable model for the data
4. **[Algorithm Development](#ml)**: Train, test, and refine various models to predict the target variable.  Given that our dependent variable `trip_duration` is a continuous outcome,  the regression algorithms to be protoyped are as follows:
 - [Multivarite Linear Regression](#linear)  


5. **[Model Deployment](#deployment)**: Apply the best performing model to the test set for contest submission.


# **ABOUT THE DATA**  
The  dataset is based on the 2022 NYC Yellow Cab trip record data. Its variables are as follows:

| **Variable Name** | **Description** | **Type**|          
| :------------------ |:-------------|:-------------|   
|vendor_id    | A code indicating the TPEP provider that provided the record.     | Number|
|pickup_datetime |  date and time when the meter was engaged|  Data & Time|
|dropoff_datetime|  date and time when the meter was disengaged|  Data & Time|
|passenger_count|  the number of passengers in the vehicle (driver entered value)|  Number|
|trip_distance |  The elapsed trip distance in miles reported by the taximeter.| Number|
|RatecodeID | The final rate code in effect at the end of the trip. | Number |
|PULocationID | TLC Taxi Zone in which the taximeter was engaged|  Number |
|DOLocationID  |   TLC Taxi Zone in which the taximeter was disengaged|  Number |
|store_and_fwd_flag | This flag indicates whether the trip record was held in vehicle memory before sending to the vendor because the vehicle did not have a connection to the server - Y=store and forward; N=not a store and forward trip   | Plain-text|
|payment_type | A numeric code signifying how the passenger paid for the trip. | Number |
|fare_amount | 	The time-and-distance fare calculated by the meter. | Number |
|extra | Miscellaneous extras and surcharges. | Number |
|mta_tax | 	Tax that is automatically triggered based on the metered rate in use.| Number |
|tip_amount | This field is automatically populated for credit card tips. Cash tips are not included. | Number|
|tolls_amount | Total amount of all tolls paid in trip. | Number |
|improvement_surcharge | Improvement surcharge assessed trips at the flag drop. The improvement surcharge began being levied in 2015. | Number |
|total_amount | The total amount charged to passengers. Does not include cash tips. | Number|
|congestion_surcharge | Total amount collected in trip for NYS congestion surcharge. | Number |
|airport_fee | For pick up only at LaGuardia and John F. Kennedy Airports. | Number |


## <a id="sampl"></a> **1.Random Sampling:**
- The original CSV file contain 3.97 milions record (more than 3.5 GB), So to make the Analysis and Modeling process more efficient we will get a random sample with (1 milion rocord) to work in it.

**I completed this step in (Random Sampling from Large CSV file.ipynb) notebook.**

## <a id="prep"></a>**2. DATA PRE-PROCESSING**
 First, the data will be loaded and cleaned into a usuable format for analysis. Specifically, I'll need to address:  
 - [missing data](#missing)
 - [outliers](#outliers)  
 - [data types](#types)
 - [feature engineering](#engineering)

In [1]:
# Import useful liberires 
import pandas as pd
import numpy as np

# Visualision Libraries
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

C:\Users\AY7\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Load our DataFrame from The Sample Data

df0 = pd.read_csv('Random_Sample_2022_Yellow_Taxi_Trip_Data.csv')
df0.shape

(998859, 20)

Our Data has (~= 1 milion record) and 20 columns 

In [3]:
# Display the first 10 rows
df0.head(10)

,Unnamed: 0,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,0,2,01/01/2022 12:40:12 AM,01/01/2022 01:07:08 AM,1.0,5.43,1.0,N,68,263,1,21.0,0.5,0.5,4.00,0.0,0.3,28.80,2.5,0.0
1,1,1,01/01/2022 12:31:53 AM,01/01/2022 12:36:31 AM,0.0,1.30,1.0,N,140,229,1,6.0,3.0,0.5,2.90,0.0,0.3,12.70,2.5,0.0
2,2,2,01/01/2022 12:43:07 AM,01/01/2022 12:51:17 AM,1.0,1.86,1.0,N,239,237,1,8.5,0.5,0.5,2.46,0.0,0.3,14.76,2.5,0.0
3,3,2,01/01/2022 12:36:17 AM,01/01/2022 12:41:48 AM,2.0,1.30,1.0,N,249,90,1,6.0,0.5,0.5,0.00,0.0,0.3,9.80,2.5,0.0
4,4,2,01/01/2022 12:55:14 AM,01/01/2022 01:11:18 AM,1.0,4.58,1.0,N,161,24,1,16.0,0.5,0.5,3.96,0.0,0.3,23.76,2.5,0.0
5,5,2,01/01/2022 01:01:24 AM,01/01/2022 01:08:03 AM,6.0,1.30,1.0,N,74,262,2,7.0,0.5,0.5,0.00,0.0,0.3,10.80,2.5,0.0
6,6,2,01/01/2022 01:01:52 AM,01/01/2022 01:11:02 AM,2.0,2.36,1.0,N,237,107,1,9.0,0.5,0.5,3.84,0.0,0.3,16.64,2.5,0.0
7,7,2,01/01/2022 12:51:14 AM,01/01/2022 01:02:45 AM,1.0,2.22,1.0,N,236,141,1,10.5,0.5,0.5,3.58,0.0,0.3,17.88,2.5,0.0
8,8,2,01/01/2022 12:55:56 AM,01/01/2022 01:17:11 AM,1.0,9.03,1.0,N,170,243,2,28.0,0.5,0.5,0.00,0.0,0.3,31.80,2.5,0.0
9,9,2,01/01/2022 12:53:34 AM,01/01/2022 12:55:13 AM,1.0,0.64,1.0,N,263,263,1,4.0,0.5,0.5,1.56,0.0,0.3,9.36,2.5,0.0


Every Row repressents a single trip.

### 2.a <a id='missing'></a> **Missing Data:**

In [4]:
# Number of missing values in each column
df0.isna().sum()

Unnamed: 0                   0
VendorID                     0
tpep_pickup_datetime         0
tpep_dropoff_datetime        0
passenger_count          34362
trip_distance                0
RatecodeID               34362
store_and_fwd_flag       34362
PULocationID                 0
DOLocationID                 0
payment_type                 0
fare_amount                  0
extra                        0
mta_tax                      0
tip_amount                   0
tolls_amount                 0
improvement_surcharge        0
total_amount                 0
congestion_surcharge     34362
airport_fee              34362
dtype: int64

In [5]:
# Number of rows have missing value/s
df0.isna().any(axis=1).sum()

34362

In [6]:
# Percentage of rows have missing value/s
print('Percentage of rows with missing value/s:',(df0.isna().any(axis=1).sum()/df0.shape[0])*100,'%')

Percentage of rows with missing value/s: 3.440125182833613 %


We have 34362 rows (3.44% of the total rows in Dataset) with missing values in (passenger_count, RatecodeID, store_and_fwd_flag, congestion_surcharge, airport_fee) columns.

In [7]:
# Investigating rows with missing values:
mask = df0.isna().any(axis=1)
df_missing_passenger_count = df0[mask]
df_missing_passenger_count.head(10)

,Unnamed: 0,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
60265,60265,2,01/01/2022 12:50:00 AM,01/01/2022 12:54:00 AM,NaN,1.00,NaN,NaN,68,246,0,13.20,0.0,0.5,1.75,0.0,0.3,18.25,NaN,NaN
60266,60266,2,01/01/2022 12:57:00 AM,01/01/2022 01:37:00 AM,NaN,14.94,NaN,NaN,238,257,0,47.13,0.0,0.5,12.41,0.0,0.3,62.84,NaN,NaN
60267,60267,2,01/01/2022 12:13:00 AM,01/01/2022 12:14:00 AM,NaN,0.28,NaN,NaN,145,145,0,13.54,0.0,0.5,1.00,0.0,0.3,15.34,NaN,NaN
60268,60268,2,01/01/2022 12:42:00 AM,01/01/2022 12:51:00 AM,NaN,1.54,NaN,NaN,142,50,0,13.20,0.0,0.5,0.88,0.0,0.3,17.38,NaN,NaN
60269,60269,2,01/01/2022 01:22:41 AM,01/01/2022 01:39:27 AM,NaN,4.18,NaN,NaN,107,263,0,13.20,0.0,0.5,2.63,0.0,0.3,19.13,NaN,NaN
60270,60270,2,01/01/2022 01:33:00 AM,01/01/2022 01:45:00 AM,NaN,2.29,NaN,NaN,148,234,0,15.02,0.0,0.5,2.93,0.0,0.3,21.25,NaN,NaN
60271,60271,2,01/01/2022 01:33:19 AM,01/01/2022 01:42:50 AM,NaN,2.26,NaN,NaN,41,239,0,13.20,0.0,0.5,3.55,0.0,0.3,20.05,NaN,NaN
60272,60272,2,01/01/2022 01:31:20 AM,01/01/2022 01:56:30 AM,NaN,7.02,NaN,NaN,232,50,0,25.66,0.0,0.5,6.28,0.0,0.3,35.24,NaN,NaN
60273,60273,2,01/01/2022 01:53:00 AM,01/01/2022 02:19:00 AM,NaN,9.13,NaN,NaN,249,227,0,28.28,0.0,0.5,8.17,0.0,0.3,39.75,NaN,NaN
60274,60274,2,01/01/2022 01:23:35 AM,01/01/2022 01:42:52 AM,NaN,5.78,NaN,NaN,75,113,0,23.13,0.0,0.5,2.88,0.0,0.3,29.31,NaN,NaN


Because the records have missing values is not a large percentage of our dataset (3.44%), and the records have missing values in 5 columns, So I will drop this records from our dataset.

In [13]:
df = df0.dropna()
df.head()

,Unnamed: 0,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,0,2,01/01/2022 12:40:12 AM,01/01/2022 01:07:08 AM,1.0,5.43,1.0,N,68,263,1,21.0,0.5,0.5,4.00,0.0,0.3,28.80,2.5,0.0
1,1,1,01/01/2022 12:31:53 AM,01/01/2022 12:36:31 AM,0.0,1.30,1.0,N,140,229,1,6.0,3.0,0.5,2.90,0.0,0.3,12.70,2.5,0.0
2,2,2,01/01/2022 12:43:07 AM,01/01/2022 12:51:17 AM,1.0,1.86,1.0,N,239,237,1,8.5,0.5,0.5,2.46,0.0,0.3,14.76,2.5,0.0
3,3,2,01/01/2022 12:36:17 AM,01/01/2022 12:41:48 AM,2.0,1.30,1.0,N,249,90,1,6.0,0.5,0.5,0.00,0.0,0.3,9.80,2.5,0.0
4,4,2,01/01/2022 12:55:14 AM,01/01/2022 01:11:18 AM,1.0,4.58,1.0,N,161,24,1,16.0,0.5,0.5,3.96,0.0,0.3,23.76,2.5,0.0


In [14]:
# Checking if their any missing values yet.
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 964497 entries, 0 to 995678
Data columns (total 20 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             964497 non-null  int64  
 1   VendorID               964497 non-null  int64  
 2   tpep_pickup_datetime   964497 non-null  object 
 3   tpep_dropoff_datetime  964497 non-null  object 
 4   passenger_count        964497 non-null  float64
 5   trip_distance          964497 non-null  float64
 6   RatecodeID             964497 non-null  float64
 7   store_and_fwd_flag     964497 non-null  object 
 8   PULocationID           964497 non-null  int64  
 9   DOLocationID           964497 non-null  int64  
 10  payment_type           964497 non-null  int64  
 11  fare_amount            964497 non-null  float64
 12  extra                  964497 non-null  float64
 13  mta_tax                964497 non-null  float64
 14  tip_amount             964497 non-nu

Now our dataset hasn't any missing values.